In [1]:
import pandas as pd
import sklearn as sfs
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('..')
from model_handler import ModelHandler
from feature_selection import FeatureSelectionAndGeneration
handler = ModelHandler()
dataset = handler.dataset
train_set = dataset[handler.train_mask]

The dataset includes different risks that need a prediction. Every risk is considered as a different target of labels, namely a response variable.

The aim is to build a model able to predict each risk in the most accurate way possible. However, the learning process is different for each of them, meaning that the minimum set of variables that best explain the largest amount of variance in the dataset is unique for every risk. As a consequence, the following pipeline will be executed as much time as the number of risks in order to return as more precise predictions as possible. 

# Dataset splitting

The first step consists in splitting the dataset into training and test sets. The first will be used during the feature selection part, which is implemented using a boosted logistic regression model. This is a supervised learning approach, thus labels are needed for the regression to be carried out. In this dataset risks are assigned to only some of the cities, therefore it's wise to select as training set all the entries containing values for the given risk. All the rest will be referred to as test set, used for the classification task, since those cities will be the ones needing a prediction.

# Feature selection

When there is a highly non-linear and complex relationship between the predictors and the labels decision trees are preferable. The dataset has many different predictors and we don't know whether this relationship is linear or not.

The most robust approach among the ensemble method is `Boosting`. It allows to aggregate many decision trees, differently from `Random Forest`, and grow them sequentially, instead of using boostrap sampling like in `Bagging`. 

The procedure consists in fitting small trees to the residuals in order to slowly improve the prediction error. Generally, model that learn slowly tend to perform better. A pitfall of Boosting, however, is that it relies very much on its tuning parameters. Hence, it's important to undergo `Cross Validation` in order to select the combination returning the highest accuracy, for every target. 
For this purpose we decided to use 3-fold cross validation in such a way to speed up the tuning process, which is already slow given the amount of parameters that need to be optimized.

In [2]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
import shutil
import os
memory_dir = '.pipeline_cache.tmp'

XgBoost has as default objective function `reg:squarederror`, which corresponds to a linear regression with mean-squared error as loss function.

In [3]:
from bayes_opt import BayesianOptimization
if os.path.isdir(memory_dir):
    shutil.rmtree(memory_dir)

def init_model(**model_params):
    return Pipeline([('generation_and_selection', FeatureSelectionAndGeneration(feats_num=500)), ('regressor', xgb.XGBRegressor(**model_params))],memory=memory_dir)
    

In [4]:
from sklearn.model_selection import cross_val_score
from data.labeled.preprocessed import RISKS_MAPPING
from classification import RANDOM_SEED
optimal_params = {}
CONSTANTS = {'objective':"reg:squarederror", "random_state": RANDOM_SEED}
for (risk, total_set, [train_set, valid_set]) in handler.get_total_train_val_set_per_risk():
    print(f"\n\n**Risk: {RISKS_MAPPING[risk]}**\n")
    print(f"Annotated Samples Size: {total_set.shape[0]}")
    print(f"To be used for parameters estimation: {train_set.shape[0]}\n")
    def xgb_evaluate(**params):
        params['n_estimators'] = int(params['n_estimators'])
        params['max_depth'] = int(params['max_depth'])
        params.update(CONSTANTS)

        model = init_model(**params)
        train_tuple = (train_set[handler.feat_names], train_set[risk])
        reg_cv = model.fit(*train_tuple)
        cv_result = np.mean(cross_val_score(model, *train_tuple, cv=3,scoring='neg_mean_squared_error'))
        return cv_result
    xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (1, 7), 
                                                 'alpha': (0,20),
                                                 'gamma': (0, 1),
                                                 'colsample_bytree': (0.3, 0.9),
                                                 "n_estimators":[200,1000],
                                                 "learning_rate":[0.1,0.5],
                                                 "min_child_weight": [0.5, 1.5],
                                                 "subsample": [0.8, 1]
                                                }
                                  
                                 )
    xgb_bo.maximize(init_points=20, n_iter=20)
    params = xgb_bo.max['params']
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params.update(CONSTANTS)
    optimal_params[risk] = params



**Risk: Higher water prices**

Annotated Samples Size: 87
To be used for parameters estimation: 60

|   iter    |  target   |   alpha   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -0.976    |  12.56    |  0.5203   |  0.353    |  0.1552   |  1.692    |  0.7748   |  321.3    |  0.9703   |
|  2        | -1.108    |  3.863    |  0.6      |  0.8569   |  0.4626   |  6.346    |  0.7124   |  420.2    |  0.8729   |
|  3        | -1.164    |  2.943    |  0.3607   |  0.2629   |  0.308    |  6.941    |  1.383    |  478.5    |  0.9804   |
|  4        | -1.027    |  15.96    |  0.7929   |  0.6658   |  0.2481   |  1.401    |  1.476    |  665.0    |  0.9819   |
|  5        | -0.9523   |  8.722    |  0.5596   |  0.8213   |  0.2037   |  3.636    |  1.121    |  834.2    |  0.8978   |
|  6        | -0.9418   |  8.082    |  0.709

|  40       | -1.052    |  5.233    |  0.4361   |  0.7784   |  0.3709   |  6.081    |  1.264    |  928.3    |  0.9005   |


**Risk: Inadequate or aging infrastructure**

Annotated Samples Size: 148
To be used for parameters estimation: 103

|   iter    |  target   |   alpha   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -0.8198   |  11.02    |  0.6243   |  0.4466   |  0.3571   |  1.014    |  0.6532   |  479.6    |  0.8418   |
|  2        | -0.8526   |  7.451    |  0.3414   |  0.4045   |  0.3367   |  4.223    |  1.107    |  722.4    |  0.9236   |
|  3        | -0.9816   |  2.071    |  0.5351   |  0.3249   |  0.1182   |  6.57     |  1.223    |  568.0    |  0.8343   |
|  4        | -0.8291   |  16.2     |  0.6908   |  0.6171   |  0.4106   |  1.451    |  0.9329   |  843.9    |  0.9049   |
|  5        | -0.8269   |  

|  39       | -0.8655   |  18.69    |  0.7914   |  0.1385   |  0.1885   |  1.891    |  0.7475   |  212.6    |  0.9122   |
|  40       | -0.8851   |  9.613    |  0.5318   |  0.01688  |  0.3891   |  6.633    |  1.372    |  221.0    |  0.9398   |


**Risk: Increased water stress or scarcity**

Annotated Samples Size: 261
To be used for parameters estimation: 182

|   iter    |  target   |   alpha   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -0.3281   |  9.485    |  0.4036   |  0.3844   |  0.1532   |  2.67     |  0.5564   |  374.0    |  0.893    |
|  2        | -0.3491   |  13.37    |  0.5434   |  0.6597   |  0.4748   |  6.06     |  0.742    |  720.1    |  0.8199   |
|  3        | -0.3278   |  8.398    |  0.4316   |  0.4138   |  0.293    |  1.407    |  0.7838   |  639.6    |  0.9499   |
|  4        | -0.3429   |  

|  38       | -0.3208   |  7.689    |  0.7648   |  0.4413   |  0.1786   |  1.792    |  1.1      |  724.1    |  0.9476   |
|  39       | -0.3405   |  7.085    |  0.6123   |  0.1234   |  0.322    |  3.815    |  0.7719   |  322.9    |  0.8477   |
|  40       | -0.3499   |  15.2     |  0.8245   |  0.1248   |  0.416    |  1.53     |  0.6271   |  654.4    |  0.9851   |


**Risk: Declining water quality**

Annotated Samples Size: 183
To be used for parameters estimation: 128

|   iter    |  target   |   alpha   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -1.076    |  11.35    |  0.4297   |  0.3311   |  0.4448   |  3.21     |  1.257    |  366.4    |  0.8716   |
|  2        | -1.461    |  0.02068  |  0.6427   |  0.2271   |  0.2137   |  2.165    |  1.089    |  850.7    |  0.8977   |
|  3        | -1.068    |  16.53    | 

|  36       | -1.126    |  6.135    |  0.8921   |  0.05173  |  0.3441   |  6.258    |  0.7195   |  311.2    |  0.9121   |
|  37       | -1.113    |  4.733    |  0.7851   |  0.7087   |  0.1775   |  6.364    |  0.6953   |  701.0    |  0.9226   |
|  38       | -1.02     |  16.33    |  0.8291   |  0.5933   |  0.3228   |  2.345    |  0.6569   |  721.1    |  0.9582   |
|  39       | -1.107    |  5.362    |  0.3      |  0.0      |  0.1029   |  1.0      |  0.5      |  720.7    |  0.8      |
|  40       | -1.04     |  19.96    |  0.5005   |  0.9126   |  0.3848   |  6.906    |  1.027    |  713.8    |  0.8768   |


**Risk: Increased water demand**

Annotated Samples Size: 98
To be used for parameters estimation: 68

|   iter    |  target   |   alpha   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -1.076    |  5.745    |  0.

|  35       | -1.273    |  2.709    |  0.5831   |  0.1776   |  0.1859   |  2.302    |  1.384    |  293.7    |  0.8151   |
|  36       | -1.038    |  10.11    |  0.6726   |  0.08261  |  0.4333   |  5.931    |  1.285    |  286.2    |  0.9211   |
|  37       | -1.099    |  5.347    |  0.7501   |  0.01408  |  0.1971   |  6.885    |  0.9091   |  282.2    |  0.8723   |
|  38       | -1.074    |  8.893    |  0.3224   |  0.8484   |  0.4658   |  1.278    |  0.942    |  283.4    |  0.9964   |
|  39       | -1.163    |  13.16    |  0.3      |  1.0      |  0.1      |  2.266    |  0.5      |  288.6    |  1.0      |
|  40       | -1.067    |  12.28    |  0.3      |  0.0      |  0.5      |  6.834    |  1.5      |  281.2    |  0.8031   |


**Risk: Regulatory**

Annotated Samples Size: 65
To be used for parameters estimation: 45

|   iter    |  target   |   alpha   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-----------------------------------------------------

|  34       | -0.65     |  7.065    |  0.9      |  0.6726   |  0.1      |  1.0      |  1.5      |  419.1    |  0.8      |
|  35       | -0.6728   |  1.945    |  0.8615   |  0.7369   |  0.2115   |  5.804    |  0.8446   |  910.2    |  0.927    |
|  36       | -0.6469   |  3.401    |  0.8743   |  0.6481   |  0.2516   |  2.601    |  0.6929   |  900.8    |  0.9294   |
|  37       | -0.9489   |  0.0      |  0.6484   |  0.0      |  0.5      |  1.0      |  1.5      |  424.1    |  0.8917   |
|  38       | -0.65     |  12.77    |  0.8338   |  0.6879   |  0.4608   |  6.81     |  1.309    |  608.4    |  0.8454   |
|  39       | -0.6375   |  6.352    |  0.414    |  0.6834   |  0.359    |  6.472    |  0.9875   |  616.6    |  0.9475   |
|  40       | -0.65     |  8.382    |  0.8514   |  0.331    |  0.1921   |  6.688    |  0.7405   |  517.9    |  0.9153   |
Using normal split for label risk6 due to underrepresentated levels. The levels counts for that label are:
 0.0    50
2.0     6
3.0     2
1.0     

|  32       | -0.5783   |  8.788    |  0.4788   |  0.1732   |  0.2638   |  4.566    |  1.347    |  568.5    |  0.9074   |
|  33       | -0.5768   |  9.276    |  0.4384   |  0.1234   |  0.1766   |  2.502    |  0.708    |  573.2    |  0.9508   |
|  34       | -0.6179   |  3.177    |  0.583    |  0.3986   |  0.4085   |  6.703    |  1.495    |  605.5    |  0.805    |
|  35       | -0.5664   |  6.128    |  0.5565   |  0.9066   |  0.3004   |  1.147    |  0.5555   |  604.5    |  0.8346   |
|  36       | -0.5517   |  6.025    |  0.8965   |  0.1584   |  0.469    |  2.295    |  1.12     |  607.4    |  0.9848   |
|  37       | -0.5847   |  8.6      |  0.6871   |  0.8713   |  0.3003   |  2.006    |  1.157    |  606.1    |  0.8259   |
|  38       | -0.6337   |  2.482    |  0.528    |  0.4878   |  0.313    |  2.417    |  1.095    |  609.5    |  0.8245   |
|  39       | -0.5801   |  7.297    |  0.4879   |  0.9732   |  0.3663   |  4.493    |  0.5141   |  607.6    |  0.9684   |
|  40       | -0.5757   

In [5]:
from data.model import MODEL_BEST_PARAMS_PATH
pd.DataFrame(optimal_params).to_csv(MODEL_BEST_PARAMS_PATH)